# Куча

Рассмотрим всем знакомую структуру данных - очередь. Это линейная последовательность данных, к которой применимы только две операции:

1. добавить элемент в конец очереди,
2. извлечь элемент из начала очереди.

Тут мы имеем фиксированный порядок элементов. Если элемент А попал в очередь раньше, чем элемент В, то и покинет очередь он раньше. Поменяем задачу: теперь мы каждому элементу сопоставим число - приоритет. Соответственно, добавляя элемент в очередь, мы протолкнем его через все элементы, приоритет которых ниже. А покидать очередь будет тот элемент, приоритет которого выше всех. Описанная выше абстрактная структура данных - очередь с приоритетами. Одной из ее реализаций является **двоичная куча** или **пирамида** (англ. *Binary heap*). Она представляет собой двоичное дерево, для которого выполняются три условия:

1. зачение в любой вершине не меньше (не больше), чем значения ее потомков;
2. на i-ом (нумерация с 0) слое $2^i$ вершин, кроме последнего;
3. последний слой заполняется слева направо без пропусков.

Используя второе свойство, оценим количество слоев в куче. Пусть в ней n элементов, тогда они распределены по слоям следующим образом (пока предположим, что n элементов хватит, чтобы заполнить нижний слой до конца): 1, 2, 4, ..., $2^i$, ..., $2^h$, где h - искомая высота. По формуле суммы первых m элементов геометрической прогрессии $S_m  = \frac{b_1 \cdot {1 - q^m}}{1 - q}$ вычислим $n = 2^{h+1}-1$ (не забываем про 0-ой слой). Тогда $\log_2{n} = \left\lfloor \log_2{(2^{h+1}-1)} \right\rfloor = h$. Отсюда получаем, что для произвольного n высота кучи оценивается как $O(\log{n})$.

![min_heap.png](min_heap.png "Пример двочиной кучи с минимумом в корне")

Самый удобный вариант хранения кучи - одномерный массив. Заметим следующую зависимость: для элемента вчейке с индексом $i$ потомки будут храниться в ячейках с индексами $2i + 1$ и $2i + 2$, а его предок будет храниться в ячейке с индексом $\left\lfloor (i - 1) / 2 \right\rfloor$.

Как и очередь куча поддерживает две операции:

1. добавить элемент в кучу,
2. извлечь из кучи максимум (минимум).

Для конкретности считаем, что на вершине куче максимум. Добавление работает следующим образом: добавим в конец кучи новый элемент X. Однако свойство 1 могло быть нарушено. Для этого посмотрим на предка нового элемента. Если он меньше, чем X, то поменяем их местами. Теперь посмотрим на нового предка X. Если свойство 1 все еще нарушается, то поменяем их местами. Это будет продолжаться? пока мы не достигнем верха или пока очередной предок будет меньше X. Заметим, что опрерация проталкивания элемента вверх выполнится не больше, чем высота кучи (так как при обмене элементов X перемещается ровно на один уровень вверх). Таким образом время работы опрерации добавления $O(\log{n})$.

Для того, чтобы выполнить извелечения максимума, необходимо последний элемент кучи поместить на самый верх, удалив максимум. После этого надо восстановить первое свойство кучи. Для этого посмотрим на двух непосредственных потомков элемента X. Если они оба меньше X, то свойство кучи восстановлено. Если хотя бы один больше X, то выберем максимального потомка и обменяем его с X. И так далее. При каждом обмене X опускается ровно на один уровень вниз, а значит что таких обменов не больше, чем высота кучи. Время работы этой операции также $O(\log{n})$.

Ниже приведен пример реализации кучи для минимума. Программа ожидает на вход одну из комманд:

+ add `value` - добавить в кучу число `value`;
+ min - извлечь минимум из кучи и напечатать его;
+ print - распечатать кучу в виде дерева;
+ exit - завершить работу программы.

In [ ]:
from math import log2


def sift_up(heap, i):
    while i > 0 and heap[(i - 1) // 2] > heap[i]:
        heap[i], heap[(i - 1) // 2] = heap[(i - 1) // 2], heap[i]
        i = (i - 1) // 2


def sift_down(heap, i):
    n = len(heap)
    while i * 2 + 1 < n:
        j = i
        if heap[i] > heap[i * 2 + 1]:
            j = i * 2 + 1
        if i * 2 + 2 < n and heap[j] > heap[i * 2 + 2]:
            j = i * 2 + 2
        if i == j:
            break
        heap[i], heap[j] = heap[j], heap[i]
        i = j


def add(heap, x):
    heap.append(x)
    sift_up(heap, len(heap) - 1)


def extract_min(heap):
    x = heap[0]
    heap[0] = heap.pop()
    sift_down(heap, 0)
    return x


def pretty_print(heap):
    height = int(log2(len(heap)))
    node_width = len(str(max(heap)))
    str_width = (2 ** (height + 1) - 1) * node_width
    interval = node_width
    result = []
    for i in range(height, -1, -1):
        start = 2 ** i - 1
        nums_in_line = 2 ** i
        args = heap[start:start + nums_in_line]
        line = (" " * interval).join(["{:" + "{}".format(node_width) + "}"] *
                                     min(nums_in_line, len(args))).format(*args)
        if i != height:
            line = " " * ((str_width - len(line)) // 2) + line
        result.append(line)
        interval = interval * 2 + node_width
    print("\n".join(reversed(result)))


if __name__ == "__main__":
    heap = []
    while True:
        cmd = input().strip()
        if cmd.find(' ') != -1:
            cmd, v = cmd.split()
            v = int(v)
        if cmd == "exit":
            break
        elif cmd == "add":
            add(heap, v)
        elif cmd == "min":
            print(extract_min(heap))
        elif cmd == "print":
            pretty_print(heap)
        else:
            print("incorrect command")